# Classify

Classification is a methodology that tries to match a text to the correct label. 

### Prompt based classification 

Prompt based classification is a methodology that relies purely on prompting the LLM in a specific way. 

### When should you use prompt based classification 

Some situations when you would use this methodology is when:
- The labels are easily understood (they don't require explanation or examples)
    
    An example is sentiment analysis
- The labels are not recognized by their semantic meaning
    
    E.g. Reasoning tasks like classifying contradictions
- You don't have many examples

### Example snippet 
Running the following code will instantiate a prompt based classifier, with a debug level for the log. 
Then it will classify the text given in "ClassifyInput".
The contents of the debuglog will be shown below.
The debuglog gives an overview of the steps taken to get the result.

In [ ]:
from os import getenv
from aleph_alpha_client import Client
from intelligence_layer.classify import SingleLabelClassify, ClassifyInput
from pprint import pprint

text_to_classify = "Beneath a canopy of stars, aliens of all kinds danced and laughed, celebrating unity in the cosmic night.\n\
The asteroid throbbed with their energy, an unforgettable space party at the heart of the galaxy."
labels = ["Space party", "Space exploration", "Something else", "Something very different and long", "Something very cool and long"]
client = Client(getenv("AA_API_TOKEN"))
task = SingleLabelClassify(client, "info")
input = ClassifyInput(
    text=text_to_classify, 
    labels=labels
)

output = task.run(input)
pprint(output.debug_log.model_dump())
pprint(output.scores)

### How does this implemetation work
For prompt based classification, we prompt the model multiple times with the text we want to classify and each of our classes. 
Instead of letting the model generate the class it thinks fits the text best, we ask it for the probability for each class.

To further explain this, lets start with a more familiar case.
The intuitive way to ask an LLM if it could label a text could be something like this: 

In [ ]:
from aleph_alpha_client import PromptTemplate

prompt_template = PromptTemplate(SingleLabelClassify.PROMPT_TEMPLATE) 
print(prompt_template.to_prompt(text=text_to_classify, label="").items[0].text)

The model would then answer our question, and give us a class that it thinks fits the text. 

In the case of classification, however, we already have the classes beforehand.
Because of this, all we are interested in is the probability the model would have guessed our specific classes.
To get this probability, we can prompt the model with each of our classes and ask the model to return the logprobs for the text. 

In case of prompt based classification the prompt looks something like this:

In [ ]:
prompt_template = PromptTemplate(SingleLabelClassify.PROMPT_TEMPLATE) 
print(prompt_template.to_prompt(text=text_to_classify, label=labels[0]).items[0].text)

As you can see, we have pre-emptively filled in the class in our prompt.

Our request will now not generate any tokens, but instead will just return us the logprobs that the class would be generated, given the previous tokens.

In [ ]:
??SingleLabelClassify._complete

In the case of the classes "Space exploration" and "Space party", the logprobs per label might look something like the code snippet below. 

In [ ]:
result_objects = [log_entry for log_entry in output.debug_log.root if log_entry.message == "Raw log probs per label"]
pprint(result_objects)

Now that we have the logprobs, we just need to do some calculations to turn them into our end score. 

To turn the logprobs into our end scores, first we normalize our probabilities. This will result in the following data structure:

In [ ]:
from typing import Optional
import graphviz  
from IPython import display
import uuid

from intelligence_layer.classify import TreeNode, Probability


normalized_probs_logs = [log_entry.value for log_entry in output.debug_log.root if log_entry.message == "Normalized Probs"]
log = normalized_probs_logs.pop()

root = TreeNode()
for probs in log.values():
    root.insert_without_calculation(probs)

def add_node(graph, node: TreeNode) -> str:
    new_node_id = str(uuid.uuid4())
    graph.node(new_node_id, label=f"{node.token.token}:{str(round(node.prob, 3))}")
    return new_node_id

def graph_path(graph, node: TreeNode, parent_id: Optional[str] = None):
    for child in node.children:
        new_node_id = add_node(graph, child)
        graph.edge(parent_id, new_node_id)
        graph_path(graph, child, new_node_id) 

def graph_nodes(root: TreeNode):
    graph = graphviz.Digraph('normalized probabilities', node_attr={'shape': 'plaintext'})
    graph.graph_attr['rankdir'] = 'TB'  
    graph.edge_attr.update(arrowhead='vee', arrowsize='1')
    for child in root.children:
        parent_id = add_node(graph, child)
        graph_path(graph, child, parent_id)
    return graph 

graph = graph_nodes(root)
display.display_svg(graph)

Finally, we take the product of all the paths to get the following results:

In [ ]:
for label, score in output.scores.items():
    print(f"{label}: {round(score, 5)}")